In [1]:
import torch
from torch import nn
import torch.autograd as autograd

from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import utils as u

In [2]:
# create all possible 8-mers
seqs8 = [''.join(x) for x in product(['A','C','G','T'], repeat=8)]
print('Total 8mers:',len(seqs8))

Total 8mers: 65536


In [3]:
# methods for assigning scores to a particular DNA sequence
score_dict = {
    'A':20,
    'C':17,
    'G':14,
    'T':11
}
def score_seqs(seqs):
    '''Each seq is just the average of the letter scores wrt score_dict'''
    data = []
    for seq in seqs:
        score = np.mean([score_dict[base] for base in seq])
        data.append([seq,score])
        
    df = pd.DataFrame(data, columns=['seq','score'])
    return df
                  
def score_seqs_motif(seqs):
    '''
    Each seq is the average of the letter scores wrt score_dict but if
    it has a TAT it gets a +10 but if it has a GCG it gets a -10
    '''
    data = []
    for seq in seqs:
        score = np.mean([score_dict[base] for base in seq])
        if 'TAT' in seq:
            score += 10
        if 'GCG' in seq:
            score -= 10
        data.append([seq,score])
        
    df = pd.DataFrame(data, columns=['seq','score'])
    return df

In [4]:
mer8 = score_seqs(seqs8)
mer8.head()

,seq,score
0,AAAAAAAA,20.000
1,AAAAAAAC,19.625
2,AAAAAAAG,19.250
3,AAAAAAAT,18.875
4,AAAAAACA,19.625


In [5]:
mer8_motif = score_seqs_motif(seqs8)
mer8_motif

,seq,score
0,AAAAAAAA,20.000
1,AAAAAAAC,19.625
2,AAAAAAAG,19.250
3,AAAAAAAT,18.875
4,AAAAAACA,19.625
...,...,...
65531,TTTTTTGT,11.375
65532,TTTTTTTA,12.125
65533,TTTTTTTC,11.750
65534,TTTTTTTG,11.375


In [6]:
# load stuff into pytorch dataloaders
mer8motif_train_dl,\
mer8motif_test_dl, \
mer8motif_train_df, \
mer8motif_test_df = u.build_dataloaders_single(mer8_motif,batch_size=11)
# change to batch size 11 so I can figure out the dimension errors

In [7]:
mer8motif_train_dl

In [8]:
print(mer8motif_train_dl.batch_size)

11


In [9]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    '''
    Apply loss function to a batch of inputs. If no optimizer
    is provided, skip the back prop step.
    '''
    print('loss batch ****')
    print("xb shape:",xb.shape)
    print("yb shape:",yb.shape)

    xb_out = model(xb.float())
    print("model out pre loss", xb_out.shape)
    loss = loss_func(xb_out, yb.float())

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    #print("lb returning:",loss.item(), len(xb))
    return loss.item(), len(xb)

def fit(epochs, model, loss_func, opt, train_dl, test_dl):
    '''
    Fit the model params to the training data, eval on unseen data.
    Loop for a number of epochs and keep train of train and val losses 
    along the way
    '''
    # keep track of losses
    train_losses = []    
    test_losses = []
    
    # loops through epochs
    for epoch in range(epochs):
        #print("TRAIN")
        model.train()
        ts = []
        ns = []
        # collect train loss; provide opt so backpropo happens
        for xb, yb in train_dl:
            t, n = loss_batch(model, loss_func, xb, yb, opt=opt)
            ts.append(t)
            ns.append(n)
        train_loss = np.sum(np.multiply(ts, ns)) / np.sum(ns)
        train_losses.append(train_loss)
        
        #print("EVAL")
        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                # loop through test batches
                # returns loss calc for test set batch size
                # unzips into two lists
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in test_dl]
                # Note: no opt provided, backprop won't happen
            )
        # Gets average MSE loss across all batches (may be of diff sizes, hence the multiply)
        #print("losses", losses)
        #print("nums", nums)
        test_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, test_loss)
        test_losses.append(test_loss)

    return train_losses, test_losses

def run_model(train_dl,test_dl, model, lr=0.01, epochs=20):
    '''
    Given data and a model type, run dataloaders with MSE loss and SGD opt
    '''
    # define loss func and optimizer
    loss_func = torch.nn.MSELoss() 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr) 
    
    # run the training loop
    train_losses, test_losses = fit(epochs, model, loss_func, optimizer, train_dl, test_dl)
    
    #return model, train_losses, test_losses
    return train_losses, test_losses

## Attempt to build LSTM

In [13]:
class DNA_LSTM(nn.Module):
    def __init__(self,seq_len,hidden_dim=10):
        super().__init__()
        self.seq_len = seq_len

        self.hidden_dim = hidden_dim
        self.hidden = None # when initialized, should be tuple of (hidden state, cell state)
        
        self.rnn = nn.LSTM(4, hidden_dim,batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
            

    
    def init_hidden(self,batch_size):
        # initialize hidden and cell states with 0s
        self.hidden =  (torch.zeros(1, batch_size, self.hidden_dim), 
                        torch.zeros(1, batch_size, self.hidden_dim))
        return self.hidden
        #hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
    

    def forward(self, xb):
        print("original xb.shape:", xb.shape)
        print(xb) # 11 x 32
        
        # make the one-hot nucleotide vectors group together
        xb = xb.view(-1,self.seq_len,4) 
        print("re-viewed xb.shape:", xb.shape) # >> 11 x 8 x 4
        print(xb)

        # ** Init hidden/cell states?? **
        batch_size = xb.shape[0]
        print("batch_size:",batch_size)
        (h,c) = self.init_hidden(batch_size)
         
        # *******
        
        lstm_out, self.hidden = self.rnn(xb, (h,c)) # should this get H and C?
        #print("lstm_out",lstm_out)
        print("lstm_out shape:",lstm_out.shape) # >> 11, 8, 10
        print("lstm_out[-1] shape:",lstm_out[-1].shape) # >> 8 x 10
        print("lstm_out[-1][-1] shape:",lstm_out[-1][-1].shape) # 10
        
        print("hidden len:",len(self.hidden)) # 2
        print("hidden[0] shape:", self.hidden[0].shape) # >> 1 x 11 x 10
        print("hidden[0][-1] shape:", self.hidden[0][-1].shape) # >> 11 X 10
        print("hidden[0][-1][-1] shape:", self.hidden[0][-1][-1].shape) # >> 10
        
        print("*****")
        # These vectors should be the same, right?
        A = lstm_out[-1][-1]
        B = self.hidden[0][-1][-1]
        print("lstm_out[-1][-1]:",A)
        print("self.hidden[0][-1][-1]",B)
        print("==?", A==B)
        print("*****")
        
        last_layer = lstm_out[-1][-1].unsqueeze(0)
        out = self.fc(last_layer) 
        print("LSTM->FC out shape:",out.shape) # what is this ?? [1]  
                                                # why?? I want it to be [11 X 1] I think??
        return out

In [14]:
seq_len = len(mer8motif_train_df['seq'].values[0])

mer8motif_model_lstm = DNA_LSTM(seq_len)
mer8motif_model_lstm

DNA_LSTM(
  (rnn): LSTM(4, 10, batch_first=True)
  (fc): Linear(in_features=10, out_features=1, bias=True)
)

In [15]:
train_losses,test_losses= run_model(
    mer8motif_train_dl,
    mer8motif_test_dl,
    mer8motif_model_lstm, 
    lr=0.01
)


loss batch ****
xb shape: torch.Size([11, 32])
yb shape: torch.Size([11, 1])
original xb.shape: torch.Size([11, 32])
tensor([[0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
         1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
         0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.],
 

/home/ewilson6/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([11, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([[[0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.]],

        [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [0., 0., 1.

self.hidden[0][-1][-1] tensor([ 0.5021,  0.8333,  0.7300,  0.3748,  0.8054, -0.7024, -0.0908, -0.2631,
        -0.1188,  0.8006], grad_fn=<SelectBackward>)
==? tensor([True, True, True, True, True, True, True, True, True, True])
*****
LSTM->FC out shape: torch.Size([1, 1])
model out pre loss torch.Size([1, 1])
loss batch ****
xb shape: torch.Size([11, 32])
yb shape: torch.Size([11, 1])
original xb.shape: torch.Size([11, 32])
tensor([[0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.,
         0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 

tensor([[[1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.]],

        [[1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.]],

        [[1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0.

...

tensor([[0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
         0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
         1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 1., 0., 1., 0., 0., 0., 

lstm_out shape: torch.Size([11, 8, 10])
lstm_out[-1] shape: torch.Size([8, 10])
lstm_out[-1][-1] shape: torch.Size([10])
hidden len: 2
hidden[0] shape: torch.Size([1, 11, 10])
hidden[0][-1] shape: torch.Size([11, 10])
hidden[0][-1][-1] shape: torch.Size([10])
*****
lstm_out[-1][-1]: tensor([ 0.8360,  0.9479,  0.9177,  0.6713,  0.9424, -0.9139, -0.1576, -0.4988,
        -0.2522,  0.9406], grad_fn=<SelectBackward>)
self.hidden[0][-1][-1] tensor([ 0.8360,  0.9479,  0.9177,  0.6713,  0.9424, -0.9139, -0.1576, -0.4988,
        -0.2522,  0.9406], grad_fn=<SelectBackward>)
==? tensor([True, True, True, True, True, True, True, True, True, True])
*****
LSTM->FC out shape: torch.Size([1, 1])
model out pre loss torch.Size([1, 1])
loss batch ****
xb shape: torch.Size([11, 32])
yb shape: torch.Size([11, 1])
original xb.shape: torch.Size([11, 32])
tensor([[0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
       

lstm_out[-1] shape: torch.Size([8, 10])
lstm_out[-1][-1] shape: torch.Size([10])
hidden len: 2
hidden[0] shape: torch.Size([1, 11, 10])
hidden[0][-1] shape: torch.Size([11, 10])
hidden[0][-1][-1] shape: torch.Size([10])
*****
lstm_out[-1][-1]: tensor([ 0.8579,  0.9548,  0.9174,  0.7368,  0.9563, -0.9080, -0.1165, -0.4794,
        -0.1854,  0.9520], grad_fn=<SelectBackward>)
self.hidden[0][-1][-1] tensor([ 0.8579,  0.9548,  0.9174,  0.7368,  0.9563, -0.9080, -0.1165, -0.4794,
        -0.1854,  0.9520], grad_fn=<SelectBackward>)
==? tensor([True, True, True, True, True, True, True, True, True, True])
*****
LSTM->FC out shape: torch.Size([1, 1])
model out pre loss torch.Size([1, 1])
loss batch ****
xb shape: torch.Size([11, 32])
yb shape: torch.Size([11, 1])
original xb.shape: torch.Size([11, 32])
tensor([[0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0., 0., 0.

*****
lstm_out[-1][-1]: tensor([ 0.8634,  0.9555,  0.8956,  0.6755,  0.9552, -0.8674, -0.1833, -0.4466,
        -0.1870,  0.9484], grad_fn=<SelectBackward>)
self.hidden[0][-1][-1] tensor([ 0.8634,  0.9555,  0.8956,  0.6755,  0.9552, -0.8674, -0.1833, -0.4466,
        -0.1870,  0.9484], grad_fn=<SelectBackward>)
==? tensor([True, True, True, True, True, True, True, True, True, True])
*****
LSTM->FC out shape: torch.Size([1, 1])
model out pre loss torch.Size([1, 1])
loss batch ****
xb shape: torch.Size([11, 32])
yb shape: torch.Size([11, 1])
original xb.shape: torch.Size([11, 32])
tensor([[0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
         0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
         0., 1., 0., 1., 0., 0., 1., 0.

KeyboardInterrupt: 